# Tweets (Sältzer)


In [ ]:
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
from studienarbeit.utils import plots

load_dotenv()

In [ ]:
FAST_MODE = False

data_dir = Path("../../data/tweets")

## Business Understanding

---

Lorem


## Data Understanding

---


### Import


In [ ]:
df = pd.read_parquet(
    data_dir / "tweets.parquet",
    columns=["screen_name", "created_at", "is_retweet", "text", "party", "birthyear", "gender"],
    use_nullable_dtypes=True,
)


In [ ]:
for col in df.columns:
    df[col] = df[col].apply(
        lambda x: None if x == "" or x == "NA" or x == "NA, NA" or x == "NA, NA, NA, NA, NA, NA, NA, NA" else x
    )


Check for missing values


In [ ]:
df.isna().sum()

In the cell above we can see that there are about 11k missing values in the `text` column. Regarding the `is_retweet` column, about 3k entries have missing values.

Following we will delete the rows.


In [ ]:
df = df.dropna(subset=["text", "is_retweet"])

Following, we can check which columns represent categorical data.


In [ ]:
df.nunique()

In [ ]:
convert_dict = {
    "screen_name": "category",
    "created_at": "datetime64[ns]",
    "is_retweet": "category",
    "text": "string",
    "party": "category",
    "birthyear": "datetime64[ns]",
    "gender": "category",
}


In [ ]:
df = df.astype(convert_dict)

In [ ]:
df.info(verbose=True, memory_usage="deep")

In [ ]:
df.describe(include="all", datetime_is_numeric=True)

In [ ]:
df.head()

### Plotting


In [ ]:
plots.plot_party_count(df)

## Data Preparation

---


In [ ]:
from pandarallel import pandarallel
from studienarbeit.utils.cleaning import CleanText
from textblob_de.sentiments import PatternAnalyzer

pandarallel.initialize(progress_bar=True, verbose=1)

### Cleaning


In [ ]:
clean = CleanText()
sentiment = PatternAnalyzer()

In [ ]:
def prep_pipeline(df: pd.DataFrame):
    # Remove tweets from parties that are not in the Bundestag
    df = df[df["party"] != "Parteilos"]

    # Group CDU and CSU as Union
    df["party"] = df["party"].replace("CSU", "UNION")
    df["party"] = df["party"].replace("CDU", "UNION")
    df["party"] = df["party"].cat.remove_unused_categories()

    # Fix labels for retweets
    df["is_retweet"] = df["is_retweet"].replace("FALSE", False)
    df["is_retweet"] = df["is_retweet"].replace("TRUE", True)
    df["is_retweet"] = df["is_retweet"].astype("bool")

    # Apply cleaning pipeline
    df["clean_text"] = df["text"].parallel_apply(lambda x: clean.clean_text(x)).astype("string")
    df["tokenized_text"] = (
        df["clean_text"].parallel_apply(lambda x: clean.remove_stopwords(clean.stemm_text(x))).astype("string")
    )

    # Count the number of words in the tweet
    df["word_count"] = df["clean_text"].parallel_apply(lambda x: len(x.split())).astype("int16")

    # Calculate the sentiment of the tweets
    # df["sentiment"] = df["clean_text"].parallel_apply(lambda x: sentiment.analyze(x).polarity).astype("float32")

    return df


Either load the cached data or process the raw tweets


In [ ]:
if FAST_MODE and (data_dir / "cache/tweets_prep.parquet").exists():
    df_prep = pd.read_parquet(data_dir / "cache/tweets_prep.parquet")
else:
    df_prep = prep_pipeline(df.copy())  # .sample(10000, random_state=42, ignore_index=True)

    if (data_dir / "cache").exists() == False:
        (data_dir / "cache").mkdir()
    df_prep.to_parquet(data_dir / "cache/tweets_prep.parquet")


In [ ]:
if (data_dir / "cache").exists() == False:
    (data_dir / "cache").mkdir()
df_prep.to_parquet(data_dir / "cache/tweets_prep_w_sent.parquet")

In [ ]:
df_prep = pd.read_parquet(data_dir / "cache/tweets_prep_wo_sent.parquet")

In [ ]:
df_prep = pd.read_parquet(data_dir / "cache/tweets_prep_w_sent.parquet")

In [ ]:
import spacy
import numpy as np
from spacy_sentiws import spaCySentiWS

In [ ]:
spacy_nlp_ger = spacy.load("de_core_news_md", exclude=["tagger", "parser", "senter", "ner"]) 
spacy_nlp_ger.add_pipe("sentiws", config={'sentiws_path': '../../data/sentiws/'})

In [ ]:
df_prep["sentiment"] = df_prep["clean_text"].parallel_apply(lambda x: np.mean([token._.sentiws if token._.sentiws != None for token in spacy_nlp_ger(x)])).astype("float32")

In [ ]:
df_prep.describe(include="all", datetime_is_numeric=True)

### Filtering


Count the number of tweets with less than 5 words


In [ ]:
print(f"Found {len(df_prep.loc[df_prep['word_count'] < 5])} tweets with less than 5 words...")

Count the number of tweets labeled as retweets

In [ ]:
print(f"The dataset contains {len(df_prep.loc[df_prep['is_retweet'] == True])} retweets.")

In [ ]:
def filtering_pipeline(df):
  df = df[df["word_count"] >= 5]
  df = df[df["is_retweet"] == False]
  
  return df

In [ ]:
df_prep = filtering_pipeline(df_prep)

### Encoding


In [ ]:
party_map = {
    "AfD": 0,
    "FDP": 1,
    "DIE GRÜNEN": 2,
    "DIE LINKE": 3,
    "SPD": 4,
    "UNION": 5,
}

df_prep["party"] = df_prep["party"].map(party_map).astype("int8")

In [ ]:
gender_map = {
    "male": 0,
    "female": 1,
}

df_prep["gender"] = df_prep["gender"].map(gender_map).astype("int8")

In [ ]:
df_prep.info(verbose=True, memory_usage="deep")

### Plotting


In [ ]:
plots.plot_party_count(df_prep)

In [ ]:
plots.plot_sentiment(df_prep)

In [ ]:
plots.plot_word_count(df_prep)

In [ ]:
plots.plot_gender(df_prep)

### Evaluation


In [ ]:
df_prep.head()

In [ ]:
df_prep.describe(include="all", datetime_is_numeric=True)

## Modeling

---


In [ ]:
df_modeling = df_prep[["clean_text", "tokenized_text", "party"]]

In [ ]:
df_modeling.to_parquet(data_dir / "cache/tweets_modeling.parquet")